# import 

In [38]:
import toml
import json
import time
import torch
import pickle
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import Module,Identity,Sequential
from typing import Dict,Tuple
from ecode.m6Anet.blocks import PoolingFilter
from sklearn.metrics import roc_auc_score

# DataLoader

In [36]:
class m6anetDataset(data.Dataset):
    def __init__(self,use_path):
        self.use_path=use_path
        self.LEN=0
        self.f_dict={}
        self.k5_list=[]
        with open(use_path+'/use_files.txt','r') as f:
            for line in f.readlines():
                f_name=line.strip()
                self.f_dict[f_name]={'len':0,'label':1,'cont':[]}
                if f_name[-4:]=='_neg':
                    self.f_dict[f_name]['label']=0
                with open(use_path+'/'+f_name+'.index') as f2:
                    for line2 in f2.readlines():
                        items2=line2.strip().split('\t')
                        self.f_dict[f_name]['cont'].append((int(items2[1]),int(items2[2])))
                        with open(self.use_path+'/'+f_name+'.json') as f:
                            f.seek(int(items2[1]),0)
                            json_str=f.read(int(items2[2])-int(items2[1]))
                            Ls=json_str.strip().split('\n')
                            k_str=json.loads(Ls[0])
                            for k5 in [k_str[9:14],k_str[10:15],k_str[11:16]]:
                                if k5 not in self.k5_list:
                                    self.k5_list.append(k5)
                self.f_dict[f_name]['len']=len(self.f_dict[f_name]['cont'])
                self.LEN+=self.f_dict[f_name]['len']
        self.k5_list=np.array(self.k5_list)
    def __getitem__(self,index):
        R_dict={'X':[],'kmer':[],'label':0}
        for key in self.f_dict:
            if index>=self.f_dict[key]['len']:
                index-=self.f_dict[key]['len']
                continue
            label=self.f_dict[key]['label']
            R_dict['label']=label
            seekit=self.f_dict[key]['cont'][index]
            with open(self.use_path+'/'+key+'.json') as f:
                f.seek(seekit[0],0)
                json_str=f.read(seekit[1]-seekit[0])
                Ls=json_str.strip().split('\n')
                k_str=json.loads(Ls[0])
                for k5 in [k_str[9:14],k_str[10:15],k_str[11:16]]:
                    R_dict['kmer'].append(np.where(self.k5_list==k5)[0][0])
                for L in Ls[1:]:
                    L_data=json.loads(L)
                    t_feature=[]
                    for each in L_data[11:14]:
                        if each[0]<0:
                            t_feature.extend([0,0,0])
                        else:
                            t_feature.extend(each)
                    R_dict['X'].append(t_feature)
            break
        for key2 in R_dict:
            R_dict[key2]=torch.tensor(R_dict[key2])
        R_dict['kmer']=R_dict['kmer'].repeat(len(R_dict['X']),1)
        return R_dict
    def __len__(self):
        return self.LEN

In [37]:
'''
[[block]]
block_type = "KmerMultipleEmbedding"
input_channel = 5722  #<--- change the .toml config file here to length of k5_list
output_channel = 2
num_neighboring_features = 1
'''
#k5 list here is from the middle 3 poses, not only the center
m5C_m6Anet_set=m6anetDataset('./edata/DataSet/m5C')
print(m5C_m6Anet_set.k5_list,len(m5C_m6Anet_set))

['AGCCA' 'GCCAC' 'CCACA' 'AGCCG' 'GCCGC' 'CCGCC' 'CGCCG' 'CCCCA' 'CCCAG'
 'CCAGC' 'GCCCC' 'CCGCG' 'CGCGG' 'GCGGC' 'GGCCC' 'CCCCC' 'GGGCC' 'CGGCC'
 'CTTCG' 'TTCGG' 'TCGGG' 'CCCCG' 'CCCGA' 'CCGAA' 'TGGCC' 'CACCG' 'ACCGC'
 'CGCCC' 'GCCCG' 'CCCAC' 'CCACC' 'GCCCA' 'CCAGG' 'CGCGC' 'GCGCC' 'GGCCG'
 'GGCCT' 'GCCTG' 'CCTGG' 'AGGCG' 'GGCGG' 'GGGCG' 'GGTCA' 'GTCAC' 'TCACT'
 'GCCCT' 'CCCTC' 'CCTCC' 'CTCCC' 'TCCCT' 'CCCTG' 'GGTCG' 'GTCGG' 'TCGGC'
 'CGGCA' 'GGCAG' 'GCAGT' 'TGGCA' 'CGGCG' 'GCGGT' 'GCCGA' 'CCGAC' 'AGACG'
 'GACGG' 'ACGGA' 'GTGCA' 'TGCAG' 'GCAGC' 'TCCCG' 'CCCGC' 'CTTCC' 'TTCCC'
 'AGACT' 'GACTT' 'ACTTT' 'CTCCT' 'TCCTC' 'TAACC' 'AACCC' 'ACCCA' 'CCCCT'
 'GAACA' 'AACAA' 'ACAAA' 'TGCCA' 'GCCAT' 'CCATG' 'GACCC' 'ACCCC' 'CCCAA'
 'AGGCT' 'GGCTG' 'GCTGG' 'TGGCT' 'GGCTT' 'GCTTA' 'CCCGT' 'CCGTC' 'GAGCC'
 'GATCA' 'ATCAC' 'CATCT' 'ATCTG' 'TCTGC' 'TCCCA' 'CCAAA' 'GCCTC' 'GAGCT'
 'AGCTG' 'GCTGT' 'TCCCC' 'CGACT' 'GACTC' 'ACTCC' 'CCGCT' 'GCTCT' 'CTCTT'
 'TCTTG' 'GCCGG' 'CCGGC' 'GGCGA' 'GCGAC' 'TGGCG' 'C

In [18]:
RELOAD=0
if RELOAD==1:
    m5C_m6Anet_set=m6anetDataset('./edata/DataSet/m5C')
    train_size=int(len(m5C_m6Anet_set)*0.8)
    test_size=len(m5C_m6Anet_set)-train_size
    m5C_m6Anet_train_set,m5C_m6Anet_test_set=torch.utils.data.random_split(m5C_m6Anet_set,[train_size,test_size])
    with open('./edata/Save_DataSet/m5C_m6Anet_train_set.pkl','wb') as f:
        pickle.dump(m5C_m6Anet_train_set,f)
    with open('./edata/Save_DataSet/m5C_m6Anet_test_set.pkl','wb') as f:
        pickle.dump(m5C_m6Anet_test_set,f)
    m5C_m6Anet_train_loader=DataLoader(m5C_m6Anet_train_set,batch_size=5,shuffle=True)
    m5C_m6Anet_test_loader=DataLoader(m5C_m6Anet_test_set,batch_size=5,shuffle=True)
else:
    with open('./edata/Save_DataSet/m5C_m6Anet_train_set.pkl','rb') as f:
        m5C_m6Anet_train_set=pickle.load(f)
    with open('./edata/Save_DataSet/m5C_m6Anet_test_set.pkl','rb') as f:
        m5C_m6Anet_test_set=pickle.load(f)
    m5C_m6Anet_train_loader=DataLoader(m5C_m6Anet_train_set,batch_size=5,shuffle=True)
    m5C_m6Anet_test_loader=DataLoader(m5C_m6Anet_test_set,batch_size=5,shuffle=True)

# model

In [19]:
class MILModel(Module):
    def __init__(self,model_config):
        super(MILModel,self).__init__()
        self.model_config=model_config
        self.read_level_encoder=None
        self.pooling_filter=None
        self.decoder=None
        self.build_model()

    def build_model(self):
        blocks=self.model_config['block']
        seq_model=[]
        for block in blocks:
            block_type=block.pop('block_type')
            block_obj=self._build_block(block_type,**block)

            if isinstance(block_obj,PoolingFilter):
                if len(seq_model)>0:
                    self.read_level_encoder=Sequential(*seq_model)
                else:
                    self.read_level_encoder=None

                self.pooling_filter=block_obj
                seq_model=[]
            else:
                seq_model.append(block_obj)

        if (self.read_level_encoder is None) and (self.pooling_filter is None):
            self.read_level_encoder=Sequential(*seq_model)
            self.pooling_filter=Identity()
            self.decoder=Identity()
        else:
            if len(seq_model)==0:
                self.decoder=Identity()
            else:
                self.decoder=Sequential(*seq_model)

    def _build_block(self,block_type,**kwargs):
        from ecode.m6Anet import blocks
        block_obj=getattr(blocks,block_type)
        return block_obj(**kwargs)

    def get_read_representation(self,x):
        if self.read_level_encoder is None:
            return x
        else:
            return self.read_level_encoder(x)

    def get_read_probability(self,x):
        read_representation=self.get_read_representation(x)
        return self.pooling_filter.predict_read_level_prob(read_representation)

    def get_site_representation(self,x):
        return self.pooling_filter(self.get_read_representation(x))

    def get_site_probability(self,x):
        return self.decoder(self.get_site_representation(x))

    def get_read_site_probability(self,x):
        read_representation=self.get_read_representation(x)
        read_level_probability=self.pooling_filter.predict_read_level_prob(read_representation)
        site_level_probability=self.decoder(self.pooling_filter(read_representation))
        return read_level_probability,site_level_probability,read_representation

    def get_attention_weights(self,x):
        if hasattr(self.pooling_filter, "get_attention_weights"):
            return self.pooling_filter.get_attention_weights(self.get_read_representation(x))
        else:
            raise ValueError("Pooling filter does not have attention weights")

    def forward(self,x):
        return self.get_site_probability(x)

# For Train and Test

In [40]:
def test(model,test_loader,device,reads_reduce=0):
    model.eval()
    right_count,all_count=0,0
    prob_all,by_all=[],[]
    with torch.no_grad():
        for _,l_dic in enumerate(test_loader):
            by=l_dic['label'].to(device)
            by=by.to(torch.int64)
            l_dic['X']=l_dic['X'][:,reads_reduce:]
            l_dic['kmer']=l_dic['kmer'][:,reads_reduce:]
            ry=model(l_dic).to(device)
            out_y=ry>0.5
            right_count+=out_y.eq(by).sum()
            all_count+=len(by)
            for each in ry:
                prob_all.append(np.array(each.cpu()))
            for each in by:
                by_all.append(np.array(each.cpu()))
    roauc=roc_auc_score(by_all,prob_all)

    accuracy=100*(right_count/all_count).item()
    print('AUC:{:.4f}   accuracy:{:.4f}%'.format(roauc,accuracy))
    torch.cuda.empty_cache()

def train(model,train_loader,test_loader,device,optimizer,loss_func,epochs,reads_reduce=0):
    torch.cuda.empty_cache()
    for epoch in range(epochs):
        total_loss=0
        model.train()
        for _,l_dic in enumerate(train_loader):
            by=l_dic['label'].to(device)
            l_dic['X']=l_dic['X'][:,reads_reduce:]
            l_dic['kmer']=l_dic['kmer'][:,reads_reduce:]
            ry=model(l_dic).to(device)
            loss=loss_func(ry,by.float())
            total_loss+=loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('epoch '+str(epoch+1)+' loss:  ',total_loss/len(test_loader))        
        if epoch%10==9:
            print('At epoch '+str(epoch+1),':')
            test(model,test_loader,device,reads_reduce)
            torch.save(model.state_dict(),'./model/model_'+str(epoch+1)+'_'+str(int(time.time()))+'.pkl')

In [41]:
def detailed_test(model,test_loader,device,reads_reduce=0,curve_name=None):
    model.eval()
    right_count,all_count=0,0
    more_dict={0.5:[0,0],0.6:[0,0],0.8:[0,0],0.7:[0,0],0.9:[0,0],0.95:[0,0],0.98:[0,0],\
               0.99:[0,0],0.995:[0,0],0.999:[0,0],0.9995:[0,0],0.9999:[0,0],0.99995:[0,0],\
               0.99999:[0,0],0.999995:[0,0],0.999999:[0,0]}
    prob_all,by_all=[],[]
    motif_found_dict={}
    motif_dict={}
    with torch.no_grad():
        for _,l_dic in enumerate(test_loader):
            by=l_dic['label'].to(device).to(torch.int64)
            l_dic['X']=l_dic['X'][:,reads_reduce:]
            l_dic['kmer']=l_dic['kmer'][:,reads_reduce:]
            ry=model(l_dic).to(device)
            out_y=ry>0.5
            right_count+=out_y.eq(by).sum()
            all_count+=len(by)
            for each in ry:
                prob_all.append(np.array(each.cpu()))
            for each in by:
                by_all.append(np.array(each.cpu()))
            for key in more_dict:
                more_dict[key][0]+=((ry>key)&by).sum()
                more_dict[key][1]+=(ry>key).sum()
    if curve_name:
        save_frame=pd.DataFrame({'label':by_all,'pred':prob_all})
        save_frame.to_csv('./edata/Save_for_drawing/'+curve_name+'_curve.csv',index=False,sep=',')

    print('Im total',all_count,'samples:')
    auc=roc_auc_score(by_all,prob_all)
    accuracy=100*(right_count/all_count).item()
    print('AUC:{:.4f}   accuracy:{:.4f}%'.format(auc,accuracy))
    for key in more_dict:
        if more_dict[key][1]>0:
            print('Precision when positive threshold at {:g} is :{:.4f}% (total:{:d})'.format(key,more_dict[key][0]/more_dict[key][1],more_dict[key][1]))
    torch.cuda.empty_cache()

# Train

In [43]:
config_file_20reads='./ecode/m6Anet/m5C_m6Anet_20reads.toml'
model_config_20reads=toml.load(config_file_20reads)
model=MILModel(model_config_20reads)
device=torch.device('cuda:0')
optimizer=optim.Adam(model.parameters(),lr=0.001)
loss_func=nn.BCELoss().to(device)
epochs=500
train(model,m5C_m6Anet_train_loader,m5C_m6Anet_test_loader,device,optimizer,loss_func,epochs,30)

epoch 1 loss:   tensor(4.6477, device='cuda:0', grad_fn=<DivBackward0>)
epoch 2 loss:   tensor(2.6624, device='cuda:0', grad_fn=<DivBackward0>)
epoch 3 loss:   tensor(2.6180, device='cuda:0', grad_fn=<DivBackward0>)
epoch 4 loss:   tensor(2.6047, device='cuda:0', grad_fn=<DivBackward0>)
epoch 5 loss:   tensor(2.5635, device='cuda:0', grad_fn=<DivBackward0>)
epoch 6 loss:   tensor(2.5130, device='cuda:0', grad_fn=<DivBackward0>)
epoch 7 loss:   tensor(2.4399, device='cuda:0', grad_fn=<DivBackward0>)
epoch 8 loss:   tensor(2.3850, device='cuda:0', grad_fn=<DivBackward0>)
epoch 9 loss:   tensor(2.3658, device='cuda:0', grad_fn=<DivBackward0>)
epoch 10 loss:   tensor(2.3224, device='cuda:0', grad_fn=<DivBackward0>)
At epoch 10 :
AUC:0.7352   accuracy:65.2402%
epoch 11 loss:   tensor(2.3024, device='cuda:0', grad_fn=<DivBackward0>)
epoch 12 loss:   tensor(2.2792, device='cuda:0', grad_fn=<DivBackward0>)
epoch 13 loss:   tensor(2.2488, device='cuda:0', grad_fn=<DivBackward0>)
epoch 14 loss: 

epoch 108 loss:   tensor(1.6372, device='cuda:0', grad_fn=<DivBackward0>)
epoch 109 loss:   tensor(1.6401, device='cuda:0', grad_fn=<DivBackward0>)
epoch 110 loss:   tensor(1.6074, device='cuda:0', grad_fn=<DivBackward0>)
At epoch 110 :
AUC:0.7619   accuracy:68.9956%
epoch 111 loss:   tensor(1.5986, device='cuda:0', grad_fn=<DivBackward0>)
epoch 112 loss:   tensor(1.6008, device='cuda:0', grad_fn=<DivBackward0>)
epoch 113 loss:   tensor(1.5950, device='cuda:0', grad_fn=<DivBackward0>)
epoch 114 loss:   tensor(1.5811, device='cuda:0', grad_fn=<DivBackward0>)
epoch 115 loss:   tensor(1.5805, device='cuda:0', grad_fn=<DivBackward0>)
epoch 116 loss:   tensor(1.5783, device='cuda:0', grad_fn=<DivBackward0>)
epoch 117 loss:   tensor(1.5724, device='cuda:0', grad_fn=<DivBackward0>)
epoch 118 loss:   tensor(1.5695, device='cuda:0', grad_fn=<DivBackward0>)
epoch 119 loss:   tensor(1.5614, device='cuda:0', grad_fn=<DivBackward0>)
epoch 120 loss:   tensor(1.5661, device='cuda:0', grad_fn=<DivBack

epoch 212 loss:   tensor(1.2101, device='cuda:0', grad_fn=<DivBackward0>)
epoch 213 loss:   tensor(1.2145, device='cuda:0', grad_fn=<DivBackward0>)
epoch 214 loss:   tensor(1.2122, device='cuda:0', grad_fn=<DivBackward0>)
epoch 215 loss:   tensor(1.2437, device='cuda:0', grad_fn=<DivBackward0>)
epoch 216 loss:   tensor(1.2146, device='cuda:0', grad_fn=<DivBackward0>)
epoch 217 loss:   tensor(1.2147, device='cuda:0', grad_fn=<DivBackward0>)
epoch 218 loss:   tensor(1.2071, device='cuda:0', grad_fn=<DivBackward0>)
epoch 219 loss:   tensor(1.2005, device='cuda:0', grad_fn=<DivBackward0>)
epoch 220 loss:   tensor(1.1955, device='cuda:0', grad_fn=<DivBackward0>)
At epoch 220 :
AUC:0.7490   accuracy:66.8996%
epoch 221 loss:   tensor(1.2150, device='cuda:0', grad_fn=<DivBackward0>)
epoch 222 loss:   tensor(1.1923, device='cuda:0', grad_fn=<DivBackward0>)
epoch 223 loss:   tensor(1.2493, device='cuda:0', grad_fn=<DivBackward0>)
epoch 224 loss:   tensor(1.2621, device='cuda:0', grad_fn=<DivBack

epoch 317 loss:   tensor(1.0717, device='cuda:0', grad_fn=<DivBackward0>)
epoch 318 loss:   tensor(1.0952, device='cuda:0', grad_fn=<DivBackward0>)
epoch 319 loss:   tensor(1.0643, device='cuda:0', grad_fn=<DivBackward0>)
epoch 320 loss:   tensor(1.1369, device='cuda:0', grad_fn=<DivBackward0>)
At epoch 320 :
AUC:0.7476   accuracy:67.7729%
epoch 321 loss:   tensor(1.1235, device='cuda:0', grad_fn=<DivBackward0>)
epoch 322 loss:   tensor(1.0593, device='cuda:0', grad_fn=<DivBackward0>)
epoch 323 loss:   tensor(1.0527, device='cuda:0', grad_fn=<DivBackward0>)
epoch 324 loss:   tensor(1.0862, device='cuda:0', grad_fn=<DivBackward0>)
epoch 325 loss:   tensor(1.0788, device='cuda:0', grad_fn=<DivBackward0>)
epoch 326 loss:   tensor(1.0737, device='cuda:0', grad_fn=<DivBackward0>)
epoch 327 loss:   tensor(1.1013, device='cuda:0', grad_fn=<DivBackward0>)
epoch 328 loss:   tensor(1.0486, device='cuda:0', grad_fn=<DivBackward0>)
epoch 329 loss:   tensor(1.0764, device='cuda:0', grad_fn=<DivBack

epoch 421 loss:   tensor(0.9299, device='cuda:0', grad_fn=<DivBackward0>)
epoch 422 loss:   tensor(0.9342, device='cuda:0', grad_fn=<DivBackward0>)
epoch 423 loss:   tensor(1.0391, device='cuda:0', grad_fn=<DivBackward0>)
epoch 424 loss:   tensor(1.0746, device='cuda:0', grad_fn=<DivBackward0>)
epoch 425 loss:   tensor(0.9209, device='cuda:0', grad_fn=<DivBackward0>)
epoch 426 loss:   tensor(0.9188, device='cuda:0', grad_fn=<DivBackward0>)
epoch 427 loss:   tensor(0.9777, device='cuda:0', grad_fn=<DivBackward0>)
epoch 428 loss:   tensor(0.9111, device='cuda:0', grad_fn=<DivBackward0>)
epoch 429 loss:   tensor(0.9229, device='cuda:0', grad_fn=<DivBackward0>)
epoch 430 loss:   tensor(0.9398, device='cuda:0', grad_fn=<DivBackward0>)
At epoch 430 :
AUC:0.7578   accuracy:69.0830%
epoch 431 loss:   tensor(0.9762, device='cuda:0', grad_fn=<DivBackward0>)
epoch 432 loss:   tensor(1.0436, device='cuda:0', grad_fn=<DivBackward0>)
epoch 433 loss:   tensor(0.9177, device='cuda:0', grad_fn=<DivBack

# Test

In [47]:
config_file_20reads='./ecode/m6anet/m5C_m6Anet_20reads.toml'
model_config_20reads=toml.load(config_file_20reads)
model=MILModel(model_config_20reads)
device=torch.device('cuda:0')
#model.load_state_dict(torch.load('./model/m5C_m6Anet_keep_model_430_20reads.pkl'))
model.load_state_dict(torch.load('./model/m5C_m6Anet_keep_model_490_20reads.pkl'))
detailed_test(model,m5C_m6Anet_test_loader,device,30,'m5C_m6Anet,20reads_3sites')

Im total 1145 samples:
AUC:0.7579   accuracy:68.2096%
Precision when positive threshold at 0.5 is :0.6732% (total:563)
Precision when positive threshold at 0.6 is :0.6870% (total:524)
Precision when positive threshold at 0.8 is :0.7200% (total:425)
Precision when positive threshold at 0.7 is :0.6981% (total:477)
Precision when positive threshold at 0.9 is :0.7398% (total:369)
Precision when positive threshold at 0.95 is :0.7651% (total:332)
Precision when positive threshold at 0.98 is :0.7886% (total:298)
Precision when positive threshold at 0.99 is :0.7986% (total:283)
Precision when positive threshold at 0.995 is :0.8130% (total:262)
Precision when positive threshold at 0.999 is :0.8230% (total:243)
Precision when positive threshold at 0.9995 is :0.8208% (total:240)
Precision when positive threshold at 0.9999 is :0.8206% (total:223)
Precision when positive threshold at 0.99995 is :0.8227% (total:220)
Precision when positive threshold at 0.99999 is :0.8238% (total:210)
Precision when 